In [5]:
## clean
!find multiplier/ ! -name 'multiplier.circom' -type f -exec rm -f {} +
!find quadratic/ ! -name 'quadratic.circom' -type f -exec rm -f {} +
!rm -rf witness.json

# Introduction to ZK (Practice)

##### ZK allows a prover to prove some statement to a verifier without revealing certain variables. For example, proving knowledge of such *m* that *hash(m) = h*, where *h* is also known to the verifier.
##### Proofs are constructed by performing some computation.
##### Then this proof can be validated using the verifier.
##### **Q**: What statements can be proven?
##### **A**: In general, anything can be proven, and specialized algorithms can probably be created for specific domains. But the most common way to express computations is through arithmetic circuits.

![Arithmetic circuit](images/1.png)

Above is an example of a simple cryptographic circuit. Circuits consist of variables and constants. They are connected by addition and multiplication operations (within field F). Variables in such computations can be public and private - public ones are known to everyone, private ones are only known to the prover and are used when creating the proof.

In the picture above, the circuit consists of two variables $x_1$, $x_2$ and constant $1$. They are connected through additions and multiplications, and ultimately result in the expression $(x_1 + x_2) \cdot (x_2 + 1) \cdot x_2$.

This simple expression doesn't carry any special meaning, however as an example we can consider it this way: let's say there is a verifier that checks if a user knows such $x_1$ and $x_2$ that the expression above equals some number $C$ (and if the check passes successfully, does something good for the prover).

Let $C = 10$. Then one of the solutions is $x_1 = 4, x_2 = 1$. Also let $x_1, x_2$ be private variables. To construct the proof, it is necessary to explicitly perform calculations in this graph using $x_1$ and $x_2$, and compare the result after calculation with constant $C$. $C$ in this case is a public input.

The verifier will receive a proof for verification (some set of numbers generated by the prover through clever zk algorithms), as well as public variables (known to both prover and verifier), and then will input 2 things into the verification function: the received proof and public variables (in this case $C$).

##### Above is a trivial scheme. But the same gates can be used to express more complex things (for example, hash functions).

##### Imagine that the verifier now checks not the knowledge of numbers $x_1$ and $x_2$, but the knowledge of such a message $m$ that $h = sha256(m)$, where $h$ is a public variable. Schematically, everything looks exactly the same as in the example above - but there are many more applications for this.

# Circom

##### Of course, in reality no one builds such schemes by hand. There are various programming languages, libraries and frameworks with different levels of abstraction for generating proofs and verifiers. Today we'll get acquainted with Circom - a fairly low-level framework for building cryptographic circuits.
#### [Documentation](https://docs.circom.io/getting-started/installation/)
##### First, let's install the necessary dependencies.

In [ ]:
!curl --proto '=https' --tlsv1.2 https://sh.rustup.rs -sSf | sh # may need to run in terminal to select options during installation

In [1]:
# !git clone https://github.com/iden3/circom.git
# !cargo build --release
# !cargo install --path circom

##### Also need to install snarkjs for validating circom scripts

In [ ]:
!npm install -g snarkjs

##### Let's verify that everything works

In [ ]:
!circom --help

##### I will duplicate code from files into cells for clarity, but we will run files from the multiplier/ directory

##### Below is the code from the file multiplier/multiplier.circom

##### Let's go through the code from top to bottom:
##### 1) On the first line we see the pragma version of the compiler - the same thing that is usually found in Solidity contracts.
##### 2) Next is a comment, it is marked with /* ... */ or // symbols and is ignored during compilation.
##### 3) Then we encounter the word **template**. Templates allow us to separate common parts of code into a separate entity. Imagine we are building a large circuit and we need to do the same multiplication many times. The most convenient option would be to put this into a template and reuse it as needed (I'll show how reuse looks later).
##### 4) In the template body, the first thing we see is the keyword **signal**. Signals are analogous to nodes in the computation graph above. After the word **signal** there can be words **input** or **output**. Signals with such keywords are visible from outside. By connecting **input** signals with other signals, we can get the **output** signal. Signals inside the template without the **input/output** keyword are not visible from outside and can only be used inside the template. The last component is the signal name, here it's *a, b, c*.
##### 5) On the next line we see a constraint (**Constraints**). These are connections for our signals. More details can be found [here](https://docs.circom.io/circom-language/constraint-generation/). In short, this construction allows us to connect signals and impose restrictions on such connections. Constraints must be represented in the form A*B + C = 0. Several syntactic constructions related to this:
##### 5.1) **a * b === C** - here **C** is some constant or another signal. Such a constraint does not connect signals, but checks that the equality is true. If during proof generation such a constraint is not satisfied, then the proof cannot be constructed.
##### 5.2) **C === a * b** - same as in 5.1, parts of equality can be swapped
##### 5.3) **x <-- a * b** - the arrow serves to create connections between signals, while this syntax does not generate and is not a constraint (!), but only assigns the new value a * b to signal x.
##### 5.4) **a * b --> x** - same as 5.3
##### 5.5) **x <== a * b** - this is a short notation for two expressions: **x <-- a * b** and **x === a * b**. This exact notation is found above in the example. Most often **<==** is used rather than **<--**. Later I'll show why **<--** can be dangerous and how it can be useful. You can write the arrow in the other direction too (**==>**).

##### As a result, we have a Multiplier2 template that takes two inputs *a* and *b*, multiplies them and places the multiplication result in signal *c*.
##### To generate proofs, we need to add an entry point for our circuit and compile it. Let's add one line to the script above:

##### The keyword component allows us to instantiate a template - here we're essentially saying that we'll have a component called main that equals Multiplier2. main has 2 inputs *a* and *b* and we get output *c*.
##### Time to compile! But what does our multiplier.circom transform into? The circom compiler generates an *r1cs constraint system*. This is a system of constraints (essentially a system of expressions in the form *D = A * B + C*) that sets restrictions for our statement. Using the code above and based on constraints (===, <==, ==>), it generates a special file that allows creating a verifier and generating proofs.
##### So, let's compile our circuit with this command:

In [ ]:
!circom multiplier/multiplier.circom --r1cs --wasm --sym --output multiplier

Hooray!
##### As output we got several files, since we used several keys during compilation:
##### 1) multiplier.r1cs - the main part, constraints in binary format. It doesn't matter what's inside, but this thing is necessary for generating the verifier and proofs.
##### 2) multiplier.sym - file for debugging TODO, but importantly it shows our signals - main.a, main.b and main.c
##### 3) multiplier_js/multiplier.wasm - file needed for witness generation - more on this below.

The next step is witness generation.
Essentially, this is a set of signals used in proof generation. To prove something, we need to assign specific values to all signals. However, most signals are actually computed from other signals, and we only need to set values for the inputs - in this case *main.a* and *main.b*.

Let's create an input.json file and specify values for our input signals in json format.

In [11]:
!cd multiplier/multiplier_js && touch input.json && echo '{"a": "3", "b": "5"}' > input.json

##### For witness generation we'll use the file ./multiplier_js/multiplier.wasm

In [12]:
!cd multiplier/multiplier_js && node generate_witness.js multiplier.wasm input.json witness.wtns

##### As output we got files witness.wtns and witness_calculator.js in the multiplier_js folder. Let's see what's inside witness.wtns. For this we need to convert this file to readable json

In [ ]:
!snarkjs wtns export json multiplier/multiplier_js/witness.wtns && cat witness.json

##### And here we can see the values of all calculated signals! The first element is always 1. Then 15 is the result, c. 3 is a, 5 is b. In total: [1, c, a, b]

Now we have everything we need to create our proof: witness.wtns and multiplier.r1cs

But what exactly are we proving? In our example, both inputs a and b are private (public access modifier needs to be explicitly added to variables, the default value is *private*). Since all our inputs are private, the verifier will only know the output of our computation. Thus, we will be proving that we know two numbers *a* and *b* such that their product equals *c*.

Trusted setup

There are many different algorithms used for generating proofs - let's take Groth16 as an example.

The first step is trusted setup. This is a procedure for generating certain parameters for creating proofs. Important: dishonest generation of such parameters can allow proving false statements, so this component of proof generation is very important.

Trusted setup consists of two parts:
1) Powers of tau procedure - this part is the same for all algorithms
2) Phase 2 - this part depends on the algorithm (in our case Groth16)

Let's start with Powers Of Tau. Let's initiate the ceremony:

In [15]:
import os
os.chdir("multiplier")

In [ ]:
!snarkjs powersoftau new bn128 12 pot12_0000.ptau -v

##### Now let's contribute to the ceremony. In a real case, many different participants contribute to the procedure - this makes the parameters secure. No one will be able to generate false proofs. In our case, we will make one contribution ourselves.

P.S. You may need to enter text as salt - this will be more convenient to do in a separate terminal.

In [ ]:
!snarkjs powersoftau contribute pot12_0000.ptau pot12_0001.ptau --name="First contribution" -v

##### We will get pot12_0001.ptau file from pot12_0000.ptau file
##### pot12_0001.ptau - our parameters from the first stage
## Phase 2

##### Let's start generating this phase:

In [ ]:
!snarkjs powersoftau prepare phase2 pot12_0001.ptau pot12_final.ptau -v

##### And a couple more commands:

In [ ]:
!snarkjs groth16 setup multiplier.r1cs pot12_final.ptau multiplier_0000.zkey

In [ ]:
!snarkjs zkey contribute multiplier_0000.zkey multiplier_0001.zkey --name="1st Contributor Name" -v

In [ ]:
!snarkjs zkey export verificationkey multiplier_0001.zkey verification_key.json

The ceremony is complete! Now we can generate the proof.
For this we will use:
1) witness.wtns - values for all signals
2) multiplier_0001.zkey - parameters from phase 2

In [21]:
!snarkjs groth16 prove multiplier_0001.zkey multiplier_js/witness.wtns proof.json public.json

##### As output we got 2 files: proof.json and public.json
##### proof.json - the proof, we'll return to it later.
##### public.json - all public signals of our circuit - in this case it's only out, the multiplication result.

In [ ]:
!cat public.json

##### Now we can verify our proof (not as a smart contract yet):

In [ ]:
!snarkjs groth16 verify verification_key.json public.json proof.json

##### As we can see, no secrets were revealed!
##### Let's do some magic and generate a verifier contract:

In [ ]:
!snarkjs zkey export solidityverifier multiplier_0001.zkey MultiplyVerifier.sol

We got the MultiplyVerifier.sol file!
It has one view function - verifyProof. As arguments it takes the previously generated proof and public inputs (in our case it's the multiplication result - 15), and returns True only if the proof is valid. We can use this in our contracts as needed!
To generate parameters for Solidity you can use this command:

In [ ]:
!snarkjs generatecall

### <-- vs <==
Why is it important to use <== instead of <-- in most places?
<-- is not actually a constraint, but merely connects signals together. This allows writing more flexible things, below is an example of checking if *in* equals zero. Returns 1 if *in == 0*, 0 otherwise.
When working with the *inv* signal, we use the **<--** operator. As we can see, the expression on the right is not in the form A * B + C, but uses a ternary operator and even division.

But for proof generation, the code itself is only used to compute witness values for signals.

Let's say we generate a proof for input 10.

The resulting witness file will look like this: **[1, out, in, inv] == [1, 0, 10, 1/10 % p]**

Nothing prevents us from substituting values in this witness. If a signal is not protected by sufficient constraints, this would allow generating a valid proof for a false input. However, if the code is written correctly, generating such a proof would be impossible.

This is exactly why the template above has the last line **in*out === 0**. It verifies that either our input is 0, or *out* is 0 - meaning our *in* is not 0 and we correctly calculated its inverse element.

Here's an example of a vulnerable implementation:

##### The code above has no constraints, so essentially any signal values in the witness will work for generating proofs. Although the code itself does what is needed, it has no effect on the proof.

More complex scheme
##### Below is an example of a scheme for generating proof that the prover knows roots of a quadratic equation.

In [ ]:
os.chdir("../quadratic")
!circom quadratic.circom --r1cs --wasm --sym

##### The example above takes an array of signals *coeffs* and signal *x* as input.
##### *Coeffs* are public inputs that will need to be provided to the verifier during verification. *x* is a hidden input and will not be known to the verifier.
##### The main entry point is *main*. There we explicitly specify that *coeffs* are public inputs.
##### *SolveQuadraticExpression* internally initializes several multiplication components: *x2*, *ax2* and *bx*. Input signals are connected to component inputs using the *<==* operator, so the line *x2.a <== x* passes signal *x* to *x2* as input *a*. When using a component, all of its **input** inputs must be initialized.
##### Component outputs can be reused: to calculate *a * x ^ 2* we pass the output of component *x2* as input (since we want to multiply coefficient *a* and *x^2*).
##### To calculate the result of the expression we use variable *var result*. **var** is not a signal and is used for simple writing of more complex expressions. Therefore, after calculating the expression value, we explicitly set the constraint: *result === 0*, meaning that *x* is a root of the expression.
##### In fact, the *Multiplier2()* components could have been omitted and everything calculated through var. Circom allows complex calculations to be carried out outside of constraints and verification, but the results of these calculations must be properly compared.
##### Here's what a simplified version could look like: